In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr

highGrowthStockDf = pd.read_csv('highGrowthStockExample.csv')
recommendedStockDf = pd.read_csv('recommendedStock.csv')

targetDf = recommendedStockDf

twoYearGainList =[]

print(targetDf['TickerSymbol'])

for i in range(len(targetDf.index)):

  stock_name = targetDf.loc[i,'TickerSymbol']
  # print(stock_name)

  # Download data from Yahoo Finance API
  stock = pdr.get_data_yahoo(stock_name,'2019-05-31')

  # Dropping Unused Columns
  stock.drop('Adj Close', axis=1, inplace=True)
  stock.drop('High', axis=1, inplace=True)
  stock.drop('Low', axis=1, inplace=True)
  stock.drop('Open', axis=1, inplace=True)
  stock.drop('Volume', axis=1, inplace=True)
  print(stock.head())

  # Get the Moving Averages for 3-day, 9-day and 21-day
  stock['3-day'] = stock['Close'].rolling(3).mean()
  stock['9-day'] = stock['Close'].rolling(9).mean()
  stock['21-day'] = stock['Close'].rolling(21).mean()

  # Get the daily changes
  stock['Change'] = np.log(stock.Close / stock.Close.shift())

  # 3-day vs 9-day MA.
  # Register 1 when 3-day MA is greater than 9-day MA and -1 when 3-day is less than 9-day
  stock['Position_3v9'] = np.where(stock['3-day'] > stock['9-day'], 1, 0)
  stock['Position_3v9'] = np.where(stock['3-day'] < stock['9-day'], -1, stock['Position_3v9'])

  # 9-day vs 21-day MA.
  # Register 1 when 9-day MA is greater than 21-day MA and -1 when 9-day is less than 21-day
  stock['Position_9v21'] = np.where(stock['9-day'] > stock['21-day'], 1,0)
  stock['Position_9v21'] = np.where(stock['9-day'] < stock['21-day'],-1,stock['Position_9v21'])

  # Calculate the volatility of the stock
  stock['Volatility_3d'] = stock.Change.rolling(3).std().shift()

  stock['System_3v9'] = np.where(stock['Position_3v9'] > 0, stock['Position_3v9'] * stock['Change'], 0)
  print(stock['System_3v9'].cumsum().tail(1)[0])
  gain = stock['System_3v9'].cumsum().tail(1)[0]
  change = stock['Change'].cumsum().tail(1)[0]
  

  twoYearGainList.append({ 'StockName': stock_name, "Gain": gain, "Change": change })

0      BTC
1     CMPS
2     CSTL
3     CURI
4     DCBO
5     GSHD
6       OM
7     UPST
8     CALT
9     DAVA
10    FLGT
11    FTHM
12    LMND
13    PUBM
14     SPT
15    TTCF
16    AVIR
17    BIGC
18    CELH
19    DOCN
20    FVRR
21    INMD
22    RETA
23    SMLR
24    YALA
25    AMWL
26    BTRS
27    DMTK
28    FROG
29     KOD
30    PGNY
31    POSH
32    SKLZ
33     RWM
34    VINP
Name: TickerSymbol, dtype: object
                 Close
Date                  
2020-10-01  100.000000
2020-10-02  100.000000
2020-10-05   99.995003
2020-10-06  100.029999
2020-10-07   99.919998
0.0032214126150717833
                Close
Date                 
2020-09-18  29.000000
2020-09-21  28.830000
2020-09-22  38.389999
2020-09-23  36.750000
2020-09-24  39.939999
0.3789849933763922
                Close
Date                 
2019-07-25  21.400000
2019-07-26  22.990000
2019-07-29  23.990000
2019-07-30  22.809999
2019-07-31  19.650000
1.6178424940650296
            Close
Date             
2020-01-08  10.0

In [2]:
print("two year gain list is: ")
print(twoYearGainList)

twoYearGainDf = pd.DataFrame(twoYearGainList)
print(twoYearGainDf)

meanDifference = (twoYearGainDf['Gain'] - twoYearGainDf['Change']).mean()
print(meanDifference)

two year gain list is: 
[{'StockName': 'BTC', 'Gain': 0.0032214126150717833, 'Change': -0.008939835411149307}, {'StockName': 'CMPS', 'Gain': 0.3789849933763922, 'Change': 0.13555517783436108}, {'StockName': 'CSTL', 'Gain': 1.6178424940650296, 'Change': 1.084494412895257}, {'StockName': 'CURI', 'Gain': 1.3916062853665483, 'Change': 0.19638882027440796}, {'StockName': 'DCBO', 'Gain': 0.5191215640669458, 'Change': 0.039605263392122086}, {'StockName': 'GSHD', 'Gain': 2.4490959883776817, 'Change': 0.9438166935243022}, {'StockName': 'OM', 'Gain': 0.21821453258813386, 'Change': -0.2754521871859464}, {'StockName': 'UPST', 'Gain': 1.784580180147487, 'Change': 1.7167374861458369}, {'StockName': 'CALT', 'Gain': 0.8387307753094428, 'Change': 0.4068247263020074}, {'StockName': 'DAVA', 'Gain': 1.4315808739867895, 'Change': 1.029388210053304}, {'StockName': 'FLGT', 'Gain': 4.360267638224032, 'Change': 2.5411004342047234}, {'StockName': 'FTHM', 'Gain': 1.6956257283675187, 'Change': 1.1316272561002638}

In [3]:
twoYearGainDf['Difference'] = twoYearGainDf['Gain'] - twoYearGainDf['Change']

In [4]:
twoYearGainDf['Difference']

0     0.012161
1     0.243430
2     0.533348
3     1.195217
4     0.479516
5     1.505279
6     0.493667
7     0.067843
8     0.431906
9     0.402193
10    1.819167
11    0.563998
12    0.672106
13    0.958984
14    0.549513
15    0.686328
16    1.389125
17    1.029141
18    0.912232
19   -0.031355
20    0.885359
21    0.582812
22    1.908894
23    0.778675
24    0.656190
25    0.979477
26    0.429892
27    3.338884
28    0.492020
29    1.116740
30    0.378153
31    0.804097
32    1.003726
33    0.914539
34    0.250327
Name: Difference, dtype: float64

In [5]:
twoYearGainDf['Difference'].mean()

0.8123881054701952

In [10]:
twoYearGainDf.to_excel('recommendedStockTwoYearGainComparison.xlsx')